In [1]:
import torch

import sys
sys.path.insert(0, './model')

import model.classifier as classifier

In [2]:
model = classifier.FaceClassifier()
print(model)

FaceClassifier(
  (model): Sequential(
    (0): Vgg_face_dag(
      (conv1_1): Conv2d(3, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
      (relu1_1): ReLU(inplace)
      (conv1_2): Conv2d(64, 64, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
      (relu1_2): ReLU(inplace)
      (pool1): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
      (conv2_1): Conv2d(64, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
      (relu2_1): ReLU(inplace)
      (conv2_2): Conv2d(128, 128, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
      (relu2_2): ReLU(inplace)
      (pool2): MaxPool2d(kernel_size=[2, 2], stride=[2, 2], padding=0, dilation=1, ceil_mode=False)
      (conv3_1): Conv2d(128, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
      (relu3_1): ReLU(inplace)
      (conv3_2): Conv2d(256, 256, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1))
      (relu3_2): ReLU(inplace)
      (conv3_3): Conv2d(256, 256, kernel_size=[3

In [4]:
# TODO - Load Data 
X = None
y = None
# X = images
# y = labels


#model.tune(X, y)

torch.save(model.state_dict(), 'model/tuned_classifier.pth')